# Notebook para el despliegue del modelo en la nube de Azure

---

In [ ]:
import os
from dotenv import load_dotenv
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.model import Model
from azureml.core import Workspace

load_dotenv()

# Usamos una variable de entorno oculta para cargar el id de la suscripción
# Se trata de un archivo .env que debe estar en el mismo directorio que este script
SUBSCRIPTION_ID = os.getenv("MY_SECRET")
TENANT_ID = os.getenv("TENANT_ID")

# Needs to be done to set the subscription id if using multiple subscriptions
# with the same tenant
ia = InteractiveLoginAuthentication(tenant_id=TENANT_ID)


ws = Workspace.create(
    name="workspace4",
    subscription_id=SUBSCRIPTION_ID,
    resource_group="act-cloud-3",
    location="eastus2",
)

MODEL_NAME = "model"
registered_model = Model.register(
    model_path="../models/linear_model.pkl",
    model_name=MODEL_NAME,
    workspace=ws,
)

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/4130bd39-7c53-419c-b1e5-8758d6d63f21/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name workspacstorage9615a8962.
Deploying KeyVault with name workspackeyvault6c3ae54a.
Deploying AppInsights with name workspacinsightsfe37e627.
Deployed AppInsights with name workspacinsightsfe37e627. Took 1.93 seconds.
Deployed KeyVault with name workspackeyvault6c3ae54a. Took 18.21 seconds.
Deploying Workspace with name workspace4.
Deployed StorageAccount with name workspacstorage9615a8962. Took 24.68 seconds.
Deployed Workspace with name workspace4. Took 21.17 seconds.
Registering model model


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

virtual_env = Environment("env-4-dates")

## Usar los paquetes correctos
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "scikit-learn"]
)

inference_config = InferenceConfig(
    environment=virtual_env,
    entry_script="../src/sscore.py",
)

aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name="service-2",
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True,
)